In [1]:
import pandas as pd
import numpy as np
#from final_model import SIRModel
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('figure', dpi=200)
import glob

In [3]:
data_path = '/Users/shivyucel/Documents/SDS_2021.nosync/SDS_2020-2021/SDS_Thesis/Data/'

In [23]:
df1 = pd.read_csv(data_path + 'h3/paper_clean_commute.csv')
df1 = df1[['SOURCE', 'TARGET', 'FLUX']]

In [ ]:

pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)

#pivoted = pivoted.loc[pivoted.index != 366]
#pivoted.drop(366, axis=1, inplace=True)

mxm_radiation = pivoted.values

h3ids = pd.read_csv('/Users/shivyucel/Documents/SDS_2021.nosync/SDS_2020-2021/SDS_Thesis/Data/h3/h3_IDs.csv')

h3ids.rename(columns={'Unnamed: 0': 'hex'}, inplace=True)


relevant = pd.DataFrame(pivoted.index)

relevant = relevant.merge(h3ids, left_on='SOURCE', right_on = '0')

pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')

merged = pop.merge(relevant, left_on='h3', right_on='hex')

merged = merged.sort_values(by='0')


populations = merged['2020_pop_h3'].values

mobility_subpops = mxm_radiation.sum(axis=1)

# Normalize the mobility matrix for each row (origin)
mobility_normalized = (mxm_radiation.T / mobility_subpops).T 

population = (mobility_normalized.T * populations).T

# Create the population-commuter table
population = np.round(population).astype(int) 

print(population.sum(axis=1).min())



In [172]:
population.shape

(2785, 2785)

In [25]:
master_lst = []
lst = []
while population.sum(axis=1).min() <= 100:

    if lst == []:
        infile = data_path + 'h3/paper_clean_commute.csv'
        df = pd.read_csv(infile)[['SOURCE', 'TARGET', 'FLUX']]
        df1 = pd.DataFrame(df.groupby(['SOURCE', 'TARGET'])['FLUX'].agg(sum))
        df1.reset_index(inplace=True)
        pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)
    else:
        pivoted.drop(index=lst, inplace=True)
        pivoted.drop(labels=lst, axis=1, inplace=True)
    
    lst = []
    mxm_radiation = pivoted.values
    
    h3ids = pd.read_csv(data_path + 'h3/commuting/h3_IDs.csv')
    
    h3ids.rename(columns={'Unnamed: 0': 'hex'}, inplace=True)
    
    
    relevant = pd.DataFrame(pivoted.index)
    
    relevant = relevant.merge(h3ids, left_on='SOURCE', right_on = '0')
    
    pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')
    
    merged = pop.merge(relevant, left_on='h3', right_on='hex')
    
    merged = merged.sort_values(by='0')
    
    populations = merged['2020_pop_h3'].values
    
    mobility_subpops = mxm_radiation.sum(axis=1)
    
    # Normalize the mobility matrix for each row (origin)
    mobility_normalized = (mxm_radiation.T / mobility_subpops).T 
    
    population = (mobility_normalized.T * populations).T
    
    # Create the population-commuter table
    population = np.round(population).astype(int) 
    
    df3 = pd.DataFrame(population, index=pivoted.index, columns=pivoted.columns)
    
    for ind, row in df3.iterrows():
        if row.sum() <= 100:
            lst.append(ind)
            master_lst.append(ind)
    
    print(population.sum(axis=1).min())

0
105


In [26]:
pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')


In [26]:
row_lst = []
lst = []
while population.sum(axis=0).min() <= 100:

    if lst == []:
        infile = data_path + 'h3/paper_clean_commute.csv'
        df = pd.read_csv(infile)[['SOURCE', 'TARGET', 'FLUX']]
        df1 = pd.DataFrame(df.groupby(['SOURCE', 'TARGET'])['FLUX'].agg(sum))
        df1.reset_index(inplace=True)
        pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)
    else:
        pivoted.drop(index=lst, inplace=True)
        pivoted.drop(labels=lst, axis=1, inplace=True)
    
    lst = []
    mxm_radiation = pivoted.values
    
    h3ids = pd.read_csv(data_path + 'h3/commuting/h3_IDs.csv')
    
    h3ids.rename(columns={'Unnamed: 0': 'hex'}, inplace=True)
    
    
    relevant = pd.DataFrame(pivoted.index)
    
    relevant = relevant.merge(h3ids, left_on='SOURCE', right_on = '0')
    
    pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')
    
    merged = pop.merge(relevant, left_on='h3', right_on='hex')
    
    merged = merged.sort_values(by='0')
    
    populations = merged['2020_pop_h3'].values
    
    mobility_subpops = mxm_radiation.sum(axis=1)

    # Normalize the mobility matrix for each row (origin)
    mobility_normalized = (mxm_radiation.T / mobility_subpops).T 
    
    population = (mobility_normalized.T * populations).T
    
    # Create the population-commuter table
    population = np.round(population).astype(int) 
    
    df3 = pd.DataFrame(population, index=pivoted.index, columns=pivoted.columns)
    
    for column in df3.columns:
        if df3[column].sum() <= 100:
            lst.append(column)
            row_lst.append(column)
    
    print(population.sum(axis=0).min())

0
16
23
91
106


In [ ]:
full_lst = master_lst + row_lst

In [35]:
df1 = df1[~df1['SOURCE'].isin(full_lst)]
df1 = df1[~df1['TARGET'].isin(full_lst)]

In [44]:
pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)

In [ ]:
merged = h3_ids.merge(pop, left_on='Unnamed: 0', right_on='h3')

In [ ]:
filtered_pop = merged[merged['0'].isin(pivoted.index)]['2020_pop_h3']

In [ ]:
#np.savetxt(data_path + 'h3/mxm_updated_real.csv', pivoted.values,  delimiter =',')

In [67]:
#np.savetxt(data_path + 'h3/population/new_h3_pop.txt', filtered_pop.values)

In [72]:
#pivoted.to_csv('correct_indices.csv')